In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from transformers import pipeline
import torch

In [ ]:
import pandas as pd

# Load dataset
data = pd.read_csv('/content/drive/MyDrive/Precog/Colab Notebooks/labels.csv')

# Preprocess text
data['text'] = data['text'].str.lower().str.replace('[^\w\s]', '', regex=True)

# Display first few rows to verify
data.head()


,Unnamed: 0,image_name,text_ocr,text,label
0,0,image_1.jpg,LOOK THERE MY FRIEND LIGHTYEAR NOW ALL SOHALIK...,look there my friend lightyear now all sohalik...,very_positive
1,1,image_2.jpeg,The best of #10 YearChallenge! Completed in le...,the best of 10 yearchallenge completed in less...,very_positive
2,2,image_3.JPG,Sam Thorne @Strippin ( Follow Follow Saw every...,sam thorne strippin follow follow saw everyon...,positive
3,3,image_4.png,10 Year Challenge - Sweet Dee Edition,10 year challenge sweet dee edition,positive
4,4,image_5.png,10 YEAR CHALLENGE WITH NO FILTER 47 Hilarious ...,10 year challenge with no filter 47 hilarious ...,neutral


In [ ]:
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import numpy as np

# Map labels to numerical values
label_mapping = {'very_positive': 0, 'positive': 1, 'neutral': 2, 'negative': 3, 'very_negative': 4}
data['label'] = data['label'].map(label_mapping)

# Handle missing values
data['label'] = data['label'].replace([np.inf, -np.inf], np.nan)  # Replace infinities
data['label'] = data['label'].fillna(-1)  # Fill NaNs with -1 or any other default value
data['label'] = data['label'].astype(int)  # Convert to integer

# Ensure texts and labels are in the correct format
texts = data['text'].astype(str).tolist()
labels = data['label'].tolist()

# Split data
train_texts, val_texts, train_labels, val_labels = train_test_split(texts, labels, test_size=0.2)

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, return_tensors='pt')
val_encodings = tokenizer(val_texts, truncation=True, padding=True, return_tensors='pt')


In [ ]:
!pip install transformers


In [ ]:
import torch
from transformers import Trainer, TrainingArguments, BertForSequenceClassification
from sklearn.metrics import accuracy_score
import numpy as np

class MemeDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
        # Check unique labels
        unique_labels = set(labels)
        print("Unique labels in dataset:", unique_labels)
        # Ensure labels are either 0 or 1
        assert all(label in [0, 1] for label in labels), "Labels must be either 0 or 1"

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Convert labels to numpy arrays for easier inspection
train_labels = np.array(train_labels.tolist())
val_labels = np.array(val_labels.tolist())

# Print unique values in the labels
print("Unique values in training labels:", np.unique(train_labels))
print("Unique values in validation labels:", np.unique(val_labels))

# Define a mapping function if you have more than two labels
def map_labels(labels):
    return [0 if label == 0 else 1 for label in labels]

# Apply mapping
train_labels = map_labels(train_labels)
val_labels = map_labels(val_labels)

# Convert back to lists
train_labels = list(train_labels)
val_labels = list(val_labels)

# Recreate datasets with corrected labels
train_dataset = MemeDataset(train_encodings, train_labels)
val_dataset = MemeDataset(val_encodings, val_labels)

# Define compute_metrics function to calculate accuracy
def compute_metrics(p):
    logits, labels = p
    predictions = logits.argmax(dim=-1)
    return {'accuracy': accuracy_score(labels, predictions)}


Unique values in training labels: [0 1 2 3 4]
Unique values in validation labels: [0 1 2 3 4]
Unique labels in dataset: {0, 1}
Unique labels in dataset: {0, 1}
